In [2]:
import base64
import mimetypes
import os
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, List, Optional

import requests
from dotenv import load_dotenv


class WordPressUploader:
    def __init__(self, site_url: str, username: str, password: str):
        """
        Initialize WordPress uploader with site credentials.
        
        Args:
            site_url: WordPress site URL (e.g., 'https://yoursite.com')
            username: WordPress username
            password: WordPress application password (not regular password)
        """
        self.site_url = site_url.rstrip('/')
        self.username = username
        self.password = password
        self.api_base = f"{self.site_url}/wp-json/wp/v2"
        
        # Create authentication header
        credentials = f"{username}:{password}"
        token = base64.b64encode(credentials.encode()).decode()
        self.headers = {
            'Authorization': f'Basic {token}',
            'Content-Type': 'application/json'
        }
    
    def test_connection(self) -> bool:
        """Test if we can connect to WordPress API."""
        try:
            response = requests.get(f"{self.api_base}/users/me", headers=self.headers)
            if response.status_code == 200:
                print("✅ Successfully connected to WordPress!")
                return True
            else:
                print(f"❌ Connection failed: {response.status_code} - {response.text}")
                return False
        except Exception as e:
            print(f"❌ Connection error: {str(e)}")
            return False
    
    def upload_image(self, image_path: str) -> Optional[Dict]:
        """
        Upload a single image to WordPress media library.
        
        Args:
            image_path: Path to the image file
            
        Returns:
            Dictionary with image info if successful, None if failed
        """
        if not os.path.exists(image_path):
            print(f"❌ Image not found: {image_path}")
            return None
        
        # Get file info
        filename = os.path.basename(image_path)
        mime_type, _ = mimetypes.guess_type(image_path)
        
        if not mime_type or not mime_type.startswith('image/'):
            print(f"❌ Not a valid image file: {filename}")
            return None
        
        try:
            # Read image file
            with open(image_path, 'rb') as f:
                image_data = f.read()
            
            # Prepare headers for media upload
            upload_headers = {
                'Authorization': self.headers['Authorization'],
                'Content-Type': mime_type,
                'Content-Disposition': f'attachment; filename="{filename}"'
            }
            
            # Upload to WordPress
            response = requests.post(
                f"{self.api_base}/media",
                headers=upload_headers,
                data=image_data
            )
            
            if response.status_code == 201:
                media_info = response.json()
                print(f"✅ Uploaded: {filename} (ID: {media_info['id']})")
                return media_info
            else:
                print(f"❌ Failed to upload {filename}: {response.status_code} - {response.text}")
                return None
                
        except Exception as e:
            print(f"❌ Error uploading {filename}: {str(e)}")
            return None
    
    def upload_images_from_folder(self, folder_path: str, extensions: List[str] = None) -> List[Dict]:
        """
        Upload all images from a folder.
        
        Args:
            folder_path: Path to folder containing images
            extensions: List of file extensions to include (default: common image formats)
            
        Returns:
            List of uploaded image info dictionaries
        """
        if extensions is None:
            extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']
        
        if not os.path.exists(folder_path):
            print(f"❌ Folder not found: {folder_path}")
            return []
        
        uploaded_images = []
        folder = Path(folder_path)
        
        # Find all image files
        image_files = []
        for ext in extensions:
            image_files.extend(folder.glob(f"*{ext}"))
            image_files.extend(folder.glob(f"*{ext.upper()}"))
        
        if not image_files:
            print(f"❌ No image files found in {folder_path}")
            return []
        
        print(f"📁 Found {len(image_files)} image(s) in folder")
        
        # Upload each image
        for image_file in sorted(image_files):
            media_info = self.upload_image(str(image_file))
            if media_info:
                uploaded_images.append(media_info)
        
        print(f"✅ Successfully uploaded {len(uploaded_images)} out of {len(image_files)} images")
        return uploaded_images
    
    def get_or_create_category(self, category_name: str) -> Optional[int]:
        """
        Get category ID by name, or create it if it doesn't exist.
        
        Args:
            category_name: Name of the category
            
        Returns:
            Category ID if successful, None if failed
        """
        try:
            # First, try to find existing category
            response = requests.get(
                f"{self.api_base}/categories",
                headers=self.headers,
                params={'search': category_name}
            )
            
            if response.status_code == 200:
                categories = response.json()
                # Look for exact match
                for cat in categories:
                    if cat['name'].lower() == category_name.lower():
                        print(f"📂 Found existing category: {category_name} (ID: {cat['id']})")
                        return cat['id']
            
            # Category doesn't exist, create it
            create_response = requests.post(
                f"{self.api_base}/categories",
                headers=self.headers,
                json={'name': category_name}
            )
            
            if create_response.status_code == 201:
                new_cat = create_response.json()
                print(f"📂 Created new category: {category_name} (ID: {new_cat['id']})")
                return new_cat['id']
            else:
                print(f"❌ Failed to create category '{category_name}': {create_response.text}")
                return None
                
        except Exception as e:
            print(f"❌ Error with category '{category_name}': {str(e)}")
            return None
    
    def get_or_create_tag(self, tag_name: str) -> Optional[int]:
        """
        Get tag ID by name, or create it if it doesn't exist.
        
        Args:
            tag_name: Name of the tag
            
        Returns:
            Tag ID if successful, None if failed
        """
        try:
            # First, try to find existing tag
            response = requests.get(
                f"{self.api_base}/tags",
                headers=self.headers,
                params={'search': tag_name}
            )
            
            if response.status_code == 200:
                tags = response.json()
                # Look for exact match
                for tag in tags:
                    if tag['name'].lower() == tag_name.lower():
                        print(f"🏷️  Found existing tag: {tag_name} (ID: {tag['id']})")
                        return tag['id']
            
            # Tag doesn't exist, create it
            create_response = requests.post(
                f"{self.api_base}/tags",
                headers=self.headers,
                json={'name': tag_name}
            )
            
            if create_response.status_code == 201:
                new_tag = create_response.json()
                print(f"🏷️  Created new tag: {tag_name} (ID: {new_tag['id']})")
                return new_tag['id']
            else:
                print(f"❌ Failed to create tag '{tag_name}': {create_response.text}")
                return None
                
        except Exception as e:
            print(f"❌ Error with tag '{tag_name}': {str(e)}")
            return None

    def create_post(self, title: str, content: str = "", status: str = "draft", 
                   featured_image_id: Optional[int] = None, categories: List[str] = None,
                   tags: List[str] = None) -> Optional[Dict]:
        """
        Create a new WordPress post.
        
        Args:
            title: Post title
            content: Post content (HTML)
            status: Post status ('draft', 'publish', 'private')
            featured_image_id: ID of image to use as featured image
            categories: List of category names to assign to the post
            tags: List of tag names to assign to the post
            
        Returns:
            Post info dictionary if successful, None if failed
        """
        post_data = {
            'title': title,
            'content': content,
            'status': status
        }
        
        if featured_image_id:
            post_data['featured_media'] = featured_image_id
        
        # Handle categories
        if categories:
            category_ids = []
            for cat_name in categories:
                cat_id = self.get_or_create_category(cat_name)
                if cat_id:
                    category_ids.append(cat_id)
            if category_ids:
                post_data['categories'] = category_ids
        
        # Handle tags
        if tags:
            tag_ids = []
            for tag_name in tags:
                tag_id = self.get_or_create_tag(tag_name)
                if tag_id:
                    tag_ids.append(tag_id)
            if tag_ids:
                post_data['tags'] = tag_ids
        
        try:
            response = requests.post(
                f"{self.api_base}/posts",
                headers=self.headers,
                json=post_data
            )
            
            if response.status_code == 201:
                post_info = response.json()
                print(f"✅ Created post: '{title}' (ID: {post_info['id']})")
                print(f"📝 Post URL: {post_info['link']}")
                return post_info
            else:
                print(f"❌ Failed to create post: {response.status_code} - {response.text}")
                return None
                
        except Exception as e:
            print(f"❌ Error creating post: {str(e)}")
            return None
    
    def create_post_with_images(self, folder_path: str, post_title: str, 
                               post_intro: str = "", post_status: str = "draft",
                               categories: List[str] = None, tags: List[str] = None) -> Optional[Dict]:
        """
        Complete workflow: upload images from folder and create post with them.
        
        Args:
            folder_path: Path to folder containing images
            post_title: Title for the blog post
            post_intro: Introduction text for the post
            post_status: Post status ('draft', 'publish', 'private')
            categories: List of category names to assign to the post
            tags: List of tag names to assign to the post
            
        Returns:
            Post info dictionary if successful, None if failed
        """
        print(f"🚀 Starting upload process for folder: {folder_path}")
        
        # Upload all images
        uploaded_images = self.upload_images_from_folder(folder_path)
        
        if not uploaded_images:
            print("❌ No images were uploaded, cannot create post")
            return None
        
        # Build post content with images
        content_parts = []
        
        if post_intro:
            content_parts.append(f"<p>{post_intro}</p>")
        
        # Add each image to the post
        for i, image_info in enumerate(uploaded_images):
            img_url = image_info['source_url']
            img_title = image_info['title']['rendered']
            img_caption = image_info.get('caption', {}).get('rendered', '')
            
            # Create image HTML
            img_html = f'<figure class="wp-block-image">'
            img_html += f'<img src="{img_url}" alt="{img_title}" />'
            if img_caption:
                img_html += f'<figcaption>{img_caption}</figcaption>'
            img_html += '</figure>'
            
            content_parts.append(img_html)
        
        full_content = '\n\n<hr />'.join(content_parts)
        
        # Use first image as featured image
        featured_image_id = uploaded_images[0]['id']
        
        # Create the post
        post_info = self.create_post(
            title=post_title,
            content=full_content,
            status=post_status,
            featured_image_id=featured_image_id,
            categories=categories,
            tags=tags
        )
        
        return post_info

## Global Nerdy Saturday picdum poster

In [ ]:

def closest_saturday():
    """
    Calculate the date of the closest upcoming Saturday.
    If today is Saturday, return today; otherwise, return the next Saturday.
    """
    today = datetime.now()
    if today.weekday() == 5:
        days_until_saturday = 0
    else:
        days_until_saturday = 5 - today.weekday()  # 5 is Saturday, 0 is Monday
        if days_until_saturday < 0 :
            days_until_saturday += 7
    return today + timedelta(days=days_until_saturday)

def title():
    saturday_text = closest_saturday().strftime("%A, %B %d")
    return f"Saturday picdump for {saturday_text}"


SITE_URL = "https://globalnerdy.com"
load_dotenv()
USERNAME = os.getenv("GLOBAL_NERDY_WORDPRESS_USERNAME")
PASSWORD = os.getenv("GLOBAL_NERDY_WORDPRESS_APP_PASSWORD")  # Use application password, not regular password
PHOTO_FOLDER = "/Users/joey/Documents/Blog/Global Nerdy/picdump GN/"
POST_TITLE = title()
POST_INTRO = "Happy Saturday, everyone! Here on Global Nerdy, Saturday means that it’s time for another “picdump” — the weekly assortment of amusing or interesting pictures, comics, and memes I found over the past week. Share and enjoy!"
POST_STATUS = "draft"  # Change to "publish" to publish immediately

# Categories and Tags
CATEGORIES = ["Picdump"]  # Will be created if they don't exist
TAGS = []  # Will be created if they don't exist

# Create uploader instance
uploader = WordPressUploader(SITE_URL, USERNAME, PASSWORD)

# Test connection
if not uploader.test_connection():
    print("❌ Cannot connect to WordPress. Check your credentials and site URL.")
    raise SystemExit(1)

# Upload photos and create post
post_info = uploader.create_post_with_images(
    folder_path=PHOTO_FOLDER,
    post_title=POST_TITLE,
    post_intro=POST_INTRO,
    post_status=POST_STATUS,
    categories=CATEGORIES,
    tags=TAGS
)

if post_info:
    print(f"\n🎉 Success! Post created with {len(post_info)} images")
    print(f"📝 Edit your post at: {post_info['link']}")
else:
    print("\n❌ Failed to create post")

✅ Successfully connected to WordPress!
🚀 Starting upload process for folder: /Users/joey/Documents/Blog/Global Nerdy/picdump GN/
📁 Found 46 image(s) in folder
✅ Uploaded: 99 bugs.jpg (ID: 44756)
✅ Uploaded: Tell me you have no friends without saying you have no friends.jpg (ID: 44757)
✅ Uploaded: blue screen ad.jpg (ID: 44758)
✅ Uploaded: built it in 2 days using copilot.jpg (ID: 44759)
✅ Uploaded: butlerian juhad.jpg (ID: 44760)
✅ Uploaded: cat 5.jpg (ID: 44761)
✅ Uploaded: code reviews.jpeg (ID: 44762)
✅ Uploaded: compiled with 0 errors.jpg (ID: 44763)
✅ Uploaded: crud.jpg (ID: 44764)
✅ Uploaded: engagement ring for engineers.jpg (ID: 44765)
✅ Uploaded: get an excel job.jpg (ID: 44766)
✅ Uploaded: girl invites you.jpg (ID: 44767)
✅ Uploaded: here for the money.jpg (ID: 44768)
✅ Uploaded: how people are using agentic ai.jpg (ID: 44769)
✅ Uploaded: how to tell if a photo is ai generated.jpg (ID: 44770)
✅ Uploaded: i fear the man.jpg (ID: 44771)
✅ Uploaded: i told you so.jpg (ID: 44772)

In [ ]:

def closest_sunday():
    """
    Calculate the date of the closest upcoming Sunday.
    If today is Sunday, return today; otherwise, return the next Sunday.
    """
    today = datetime.now()
    if today.weekday() == 6:
        days_until_sunday = 0
    else:
        days_until_sunday = 6 - today.weekday()  # 5 is Saturday, 0 is Monday
        if days_until_sunday < 0 :
            days_until_sunday += 7
    return today + timedelta(days=days_until_sunday)

def title():
    sunday_text = closest_sunday().strftime("%A, %B %-d")
    return f"Sunday picdump for {sunday_text}"


SITE_URL = "https://joeydevilla.com"
load_dotenv()
USERNAME = os.getenv("ACCORDION_GUY_WORDPRESS_USERNAME")
PASSWORD = os.getenv("ACCORDION_GUY_WORDPRESS_APP_PASSWORD")  # Use application password, not regular password
PHOTO_FOLDER = "/Users/joey/Documents/Blog/Accordion Guy/picdump AG/"
POST_TITLE = title()
POST_INTRO = """<p>It’s Sunday, and it’s time for another “picdump!” Here are the memes, pictures, and cartoons floating
around the internet that I found interesting or relevant this week. Share and enjoy!</p>"""
POST_STATUS = "draft"  # Change to "publish" to publish immediately

# Categories and Tags
CATEGORIES = ["Picdump", "Spicy"]  # Will be created if they don't exist
TAGS = []  # Will be created if they don't exist

# Create uploader instance
uploader = WordPressUploader(SITE_URL, USERNAME, PASSWORD)

# Test connection
if not uploader.test_connection():
    print("❌ Cannot connect to WordPress. Check your credentials and site URL.")
    raise SystemExit(1)

# Upload photos and create post
post_info = uploader.create_post_with_images(
    folder_path=PHOTO_FOLDER,
    post_title=POST_TITLE,
    post_intro=POST_INTRO,
    post_status=POST_STATUS,
    categories=CATEGORIES,
    tags=TAGS
)

if post_info:
    print(f"\n🎉 Success! Post created with {len(post_info)} images")
    print(f"📝 Edit your post at: {post_info['link']}")
else:
    print("\n❌ Failed to create post")


✅ Successfully connected to WordPress!
🚀 Starting upload process for folder: /Users/joey/Documents/Blog/Accordion Guy/picdump AG/
📁 Found 178 image(s) in folder
✅ Uploaded: 4 seasons in florida.JPG (ID: 122528)
✅ Uploaded: 400 years ago wasnt great either.jpg (ID: 122529)
✅ Uploaded: a felon married to an immigrant.jpg (ID: 122530)
✅ Uploaded: aliigator alcatraz merch.JPG (ID: 122531)
✅ Uploaded: alligator alcatraz is a modern-day lynching joke.jpg (ID: 122532)
✅ Uploaded: alligator alcatraz.jpg (ID: 122533)
✅ Uploaded: alligator auschwitz seems more fitting.jpg (ID: 122534)
✅ Uploaded: alligator auschwitz.jpg (ID: 122535)
✅ Uploaded: amendment to give veterans jobs back.JPG (ID: 122536)
✅ Uploaded: america is beautiful contradictory unfinished.jpg (ID: 122537)
✅ Uploaded: american who dont want to fight.JPG (ID: 122538)
❌ Failed to upload americans seeking healthcare in rural hospita;s 2026-27.jpg: 500 - {"code":"rest_upload_sideload_error","message":"Sorry, you are not allowed to upl